In [1]:
# import the library
from pynq import Overlay     # import the overlay
from pynq import allocate    # import for CMA (contingeous memory allocation)
from pynq import DefaultIP   # import the ip connector library for extension
from pynq import Interrupt
import pynq_driver
import asyncio
import numpy as np
import os
import subprocess
import re
import time

In [2]:
overlay = Overlay("system.bit")
help(overlay)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay system.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    myproject_gem_1      : pynq_driver.MyDfxCtrl
    axi_intc_0           : pynq.overlay.DefaultIP
    ps_e                 : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [3]:

overlay.interrupt_pins

{'irq_const_tieoff/dout': {'controller': 'axi_intc_0',
  'index': 31,
  'fullpath': 'irq_const_tieoff/dout'},
 'axi_intc_0_intr_1_interrupt_concat/In0': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'axi_intc_0_intr_1_interrupt_concat/In0'},
 'myproject_gem_1/interrupt': {'controller': 'axi_intc_0',
  'index': 1,
  'fullpath': 'myproject_gem_1/interrupt'},
 'axi_intc_0_intr_1_interrupt_concat/In1': {'controller': 'axi_intc_0',
  'index': 1,
  'fullpath': 'axi_intc_0_intr_1_interrupt_concat/In1'},
 'axi_intc_0_intr_1_interrupt_concat/In2': {'controller': 'axi_intc_0',
  'index': 2,
  'fullpath': 'axi_intc_0_intr_1_interrupt_concat/In2'},
 'axi_intc_0_intr_1_interrupt_concat/In3': {'controller': 'axi_intc_0',
  'index': 3,
  'fullpath': 'axi_intc_0_intr_1_interrupt_concat/In3'},
 'axi_intc_0_intr_1_interrupt_concat/In4': {'controller': 'axi_intc_0',
  'index': 4,
  'fullpath': 'axi_intc_0_intr_1_interrupt_concat/In4'},
 'axi_intc_0_intr_1_interrupt_concat/In5': {'controller': 

In [4]:
# create an instance of the interrupt
my_interrupt = Interrupt('myproject_gem_1/interrupt')

In [5]:
# Load input from .npy file
input_array  = np.load("inputGenbit.npy").astype(np.float32)  # shape (20,4,4,1)
output_array = np.zeros(input_array.shape, dtype=np.float32)

# Allocate physically contiguous memory for input and output
input_buffer = allocate(shape=input_array.shape, dtype=np.float32)
output_buffer = allocate(shape=output_array.shape, dtype=np.float32)

In [6]:
# check input shape
print(f"input array shape {input_array.shape}")
print(f"output array shape {output_array.shape}")

input array shape (10000, 4, 4, 1)
output array shape (10000, 4, 4, 1)


In [7]:
# copy data to input buffer
np.copyto(input_buffer, input_array)
input_buffer.flush()

In [8]:
# get the ip and initialize the system
ip = overlay.myproject_gem_1  # Replace with your IP instance name
ip.set_input (0, input_buffer)
ip.set_output(0, output_buffer)
ip.set_amt_query(input_array.shape[0])
ip.prepare_intr()

input gmem_in0_ptr_input_1 will be set to addr: 0x78400000 with elements: 160000
output gmem_out0_ptr_layer12_out will be set to addr: 0x78500000 with elements: 160000
amount of queries will be set to: 10000 at address: 0x28
prepare your interrupt
global interrupt enable register
enable gie successful
ap_done interrupt enable register
enable ap_done interrupt successful
ap_done register clear
clear ap_done interrupt successful
----------------------


In [9]:
async def wait_for_acc():
    print("starting the accelerator")
    ip.ctrl_start()
    print("waiting for the accelerator to finish")
    await my_interrupt.wait()
    print("accelerator has finished")


In [10]:
#### get event loop from asyncio
loop = asyncio.get_event_loop()

In [11]:
task = loop.create_task(wait_for_acc())
loop.run_until_complete(task)

starting the accelerator
waiting for the accelerator to finish
accelerator has finished


In [12]:
output_buffer.invalidate()

In [13]:
print(output_buffer)

[[[[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.49609375]]

  [[0.5       ]
   [0.5       ]
   [0.4921875 ]
   [0.4765625 ]]]


 [[[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.48046875]
   [0.48046875]]]


 [[[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.49609375]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.48828125]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.4921875 ]]]


 ...


 [[[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.48046875]

In [14]:
# convert it to numpy array
print("we got output shape:", output_buffer.shape)
outNp = np.array(output_buffer)

we got output shape: (10000, 4, 4, 1)


In [15]:
# save it to .npy file
np.save("out_hw.npy", outNp)